##Importar librerias

In [1]:
import numpy as np 
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
%matplotlib inline
plt.style.use('ggplot')
pd.set_option('display.max_rows', 500)
pd.options.display.float_format = '{:,.2f}'.format

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##Importo tres csv nuevos
1- Sociodemographic: df original 
2- region_file : Contiene las regiones de España con su respectivo codigo
3- pob_esp_file : Contiene las poblaciones de españa y su region code
4- pop_country : Contiene la poblacion total de los paises y su country_id
5- pop_region_oc_file : Contiene la contiene las regiones mas pobladas y los country_id de los que no tienen region por no ser de España

In [5]:
sd_file = '/content/drive/MyDrive/CAPSTONE/sociodemographic_df.csv'

In [6]:
region_file = '/content/drive/MyDrive/CAPSTONE/codprov.csv'

In [7]:
pob_esp_file = '/content/drive/MyDrive/CAPSTONE/poblacion_x_region_españa_2021.csv'

In [8]:
pop_country = '/content/drive/MyDrive/CAPSTONE/population_country.csv'

In [9]:
pop_region_oc_file = '/content/drive/MyDrive/CAPSTONE/region_code_oc.csv'

In [10]:
region_code = pd.read_csv(region_file, ';')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


In [11]:
sociodemographic = pd.read_csv(sd_file)

In [12]:
pop_esp = pd.read_csv(pob_esp_file, ';')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


In [13]:
pop_country = pd.read_csv(pop_country, ';')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


In [14]:
pop_region_oc = pd.read_csv(pop_region_oc_file , ';')


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


##Country_id




###Con el country_id vamos a agregar una columna con la poblacion de todos los paises

In [15]:
#hay clientes de 41 paises
sociodemographic['country_id'].nunique()

41

In [16]:
#la mayoria de los clientes tienen residencia en españa. podriamos ver que peso tienen los clientes de los demas paises en la adquisicion de productos
sociodemographic['country_id'].value_counts()

ES    5960672
GB        441
FR        225
DE        199
US        195
CH        194
BR         87
BE         81
VE         79
IE         68
MX         58
AT         51
AR         51
PL         49
IT         45
MA         34
CL         30
CN         28
CA         22
LU         17
ET         17
QA         17
CI         17
SA         17
CM         17
SN         17
MR         17
NO         17
RU         17
CO         17
GA         17
GT         17
DO         17
SE         16
DJ         11
PT         11
JM         11
RO          9
HU          8
DZ          7
PE          4
Name: country_id, dtype: int64

In [ ]:
sociodemographic[sociodemographic["region_code"].isna()] ["country_id"].value_counts().sum()

2264

In [ ]:
#estos son los paises que tienen nulos en region code 
sociodemographic[sociodemographic["region_code"].isna()] ["country_id"].value_counts().sum()

2264

In [ ]:
#vemos cuales son los paises que no tienen nulos en region
sociodemographic[sociodemographic["region_code"].notnull()] ["country_id"].value_counts()

ES    5960660
Name: country_id, dtype: int64

In [ ]:
#vemos si este cliente tiene regiones nulas
sociodemographic[sociodemographic["pk_cid"] ==1234433] ["region_code"]

215020      NaN
276408      NaN
517930      NaN
760234      NaN
1190279     NaN
1442145     NaN
1744982     NaN
2113491     NaN
2256273     NaN
2864559     NaN
3044630     NaN
3483305     NaN
4057071   12.00
4281599   12.00
4962695   12.00
5412371   12.00
5904045   12.00
Name: region_code, dtype: float64

In [ ]:
#lo rellenamos con el valor que ya tenía ese cliente para la región
sociodemographic.loc[(sociodemographic["country_id"] == "ES") & (sociodemographic["pk_cid"] ==1234433), "region_code"] = 12.00

In [ ]:
sociodemographic[sociodemographic["pk_cid"] ==1234433] ["region_code"]

215020    12.00
276408    12.00
517930    12.00
760234    12.00
1190279   12.00
1442145   12.00
1744982   12.00
2113491   12.00
2256273   12.00
2864559   12.00
3044630   12.00
3483305   12.00
4057071   12.00
4281599   12.00
4962695   12.00
5412371   12.00
5904045   12.00
Name: region_code, dtype: float64

##Region code

In [ ]:
region_code.head()

,CODIGO,LITERAL
0,2,Albacete
1,3,Alicante
2,4,Almería
3,1,Araba
4,33,Asturias


In [ ]:
country_id = pd.DataFrame(sociodemographic['country_id'])

In [ ]:
pop_esp.head()

,Region_code,Year,Population_rEs
0,1,2021,333626
1,2,2021,386464
2,3,2021,1881762
3,4,2021,731792
4,5,2021,158421


In [19]:
sociodemographic['region_code'].nunique()

52

In [20]:
#aqui hago el merge del df sdemo con el de la poblacion x region usando el region code
sociodemographic_merge = sociodemographic.merge(pop_esp[['Region_code', 'Population_rEs']], how= 'left', left_on= 'region_code', right_on= 'Region_code')

In [21]:
sociodemographic_merge = sociodemographic_merge.merge(region_code[['LITERAL', 'CODIGO']], how= 'left', left_on= 'region_code', right_on= 'CODIGO')

In [22]:
#veo que se agrego la columna de region code (tengo que borrarla) y la de population x region que queria
sociodemographic_merge

,Unnamed: 0,pk_cid,pk_partition,country_id,region_code,gender,age,deceased,salary,Region_code,Population_rEs,LITERAL,CODIGO
0,0,1375586,2018-01-28,ES,29.00,H,35,N,"87,218.10",29.00,"1,695,651.00",Málaga,29.00
1,1,1050611,2018-01-28,ES,13.00,V,23,N,"35,548.74",13.00,"492,591.00",Ciudad Real,13.00
2,2,1050612,2018-01-28,ES,13.00,V,23,N,"122,179.11",13.00,"492,591.00",Ciudad Real,13.00
3,3,1050613,2018-01-28,ES,50.00,H,22,N,"119,775.54",50.00,"967,452.00",Zaragoza,50.00
4,4,1050614,2018-01-28,ES,50.00,V,23,N,NaN,50.00,"967,452.00",Zaragoza,50.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5962919,13647304,1166765,2019-05-28,ES,50.00,V,22,N,"43,912.17",50.00,"967,452.00",Zaragoza,50.00
5962920,13647305,1166764,2019-05-28,ES,26.00,V,23,N,"23,334.99",26.00,"319,796.00",La Rioja,26.00
5962921,13647306,1166763,2019-05-28,ES,50.00,H,47,N,NaN,50.00,"967,452.00",Zaragoza,50.00
5962922,13647307,1166789,2019-05-28,ES,50.00,H,22,N,"199,592.82",50.00,"967,452.00",Zaragoza,50.00


In [23]:
sociodemographic_merge['region_code'].value_counts()

28.00    1185757
8.00      600362
46.00     358142
30.00     296856
41.00     290447
15.00     254891
29.00     187911
36.00     175850
3.00      171358
11.00     160894
33.00     134562
50.00     132983
6.00      122587
35.00     121982
47.00     116948
18.00     102789
45.00     102328
37.00      92538
10.00      83283
14.00      75405
2.00       72071
21.00      72037
13.00      70092
39.00      69589
12.00      68709
7.00       62436
32.00      54639
27.00      52783
43.00      51782
17.00      50749
9.00       47746
25.00      46887
16.00      36094
24.00      35426
48.00      35256
26.00      34790
4.00       32680
31.00      31241
49.00      30965
23.00      30955
38.00      29001
34.00      27545
19.00      26762
40.00      21658
5.00       21300
22.00      21112
20.00      18833
1.00       12999
44.00      12616
42.00       8500
51.00       2896
52.00       2638
Name: region_code, dtype: int64

In [ ]:
#borro la columna de region que sobra
sociodemographic_merge.drop('Region_code', axis=1, inplace=True)

In [ ]:
sociodemographic_merge.drop('CODIGO', axis=1, inplace=True)

In [ ]:
sociodemographic_merge.rename(columns={"LITERAL": "Region_name1"},inplace=True)

In [ ]:
#ahora voy a ver cuales son los paises que no tienen region para asignarle la poblacion de la region mas reelevante de su pais
sociodemographic_merge['country_id'].value_counts()

ES    5960672
GB        441
FR        225
DE        199
US        195
CH        194
BR         87
BE         81
VE         79
IE         68
MX         58
AT         51
AR         51
PL         49
IT         45
MA         34
CL         30
CN         28
CA         22
LU         17
ET         17
QA         17
CI         17
SA         17
CM         17
SN         17
MR         17
NO         17
RU         17
CO         17
GA         17
GT         17
DO         17
SE         16
DJ         11
PT         11
JM         11
RO          9
HU          8
DZ          7
PE          4
Name: country_id, dtype: int64

####Población por pais

In [ ]:
pop_region_oc

,country_id,Population_Roc,Region_name
0,GB,9002488,London
1,FR,12324261,Ill de france
2,DE,17925570,North Rhine-Westphalia
3,US,39613493,California
4,CH,1487969,Zurich
5,BR,46289333,Sao Paulo
6,BE,1869730,Flemish Region
7,VE,4957765,Zulia
8,MA,4270750,Casablanca
9,IE,2630720,Leinster


In [ ]:
#merge por country id con ISO 02 para obtener la columna de poblacion en el dataframe. Tomado del banco mundial
sociodemographic_merge= sociodemographic_merge.merge(pop_country[['2char country code', 'Population_country']], how= 'left', left_on= 'country_id', right_on= '2char country code')

In [ ]:
sociodemographic_merge

,Unnamed: 0,pk_cid,pk_partition,country_id,region_code,gender,age,deceased,salary,Population_rEs,Region_name1,2char country code,Population_country
0,0,1375586,2018-01-28,ES,29.00,H,35,N,"87,218.10","1,695,651.00",Málaga,ES,"46,754,778.00"
1,1,1050611,2018-01-28,ES,13.00,V,23,N,"35,548.74","492,591.00",Ciudad Real,ES,"46,754,778.00"
2,2,1050612,2018-01-28,ES,13.00,V,23,N,"122,179.11","492,591.00",Ciudad Real,ES,"46,754,778.00"
3,3,1050613,2018-01-28,ES,50.00,H,22,N,"119,775.54","967,452.00",Zaragoza,ES,"46,754,778.00"
4,4,1050614,2018-01-28,ES,50.00,V,23,N,NaN,"967,452.00",Zaragoza,ES,"46,754,778.00"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5962919,13647304,1166765,2019-05-28,ES,50.00,V,22,N,"43,912.17","967,452.00",Zaragoza,ES,"46,754,778.00"
5962920,13647305,1166764,2019-05-28,ES,26.00,V,23,N,"23,334.99","319,796.00",La Rioja,ES,"46,754,778.00"
5962921,13647306,1166763,2019-05-28,ES,50.00,H,47,N,NaN,"967,452.00",Zaragoza,ES,"46,754,778.00"
5962922,13647307,1166789,2019-05-28,ES,50.00,H,22,N,"199,592.82","967,452.00",Zaragoza,ES,"46,754,778.00"


In [ ]:
#veo si funciona
sociodemographic_merge[sociodemographic_merge['country_id'] != 'ES']

,Unnamed: 0,pk_cid,pk_partition,country_id,region_code,gender,age,deceased,salary,Population_rEs,Region_name1,2char country code,Population_country
2022,2031,1050073,2018-01-28,CA,NaN,V,53,N,NaN,NaN,NaN,CA,"37,742,154.00"
4121,4139,1052929,2018-01-28,CH,NaN,V,34,N,NaN,NaN,NaN,CH,"8,654,622.00"
4320,4342,1054714,2018-01-28,CL,NaN,V,40,N,NaN,NaN,NaN,CL,"19,116,201.00"
5874,5907,1044441,2018-01-28,IE,NaN,V,28,N,NaN,NaN,NaN,IE,"4,937,786.00"
15144,15222,1058997,2018-01-28,AT,NaN,V,43,N,NaN,NaN,NaN,AT,"9,006,398.00"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5958237,13642621,1173247,2019-05-28,DJ,NaN,V,40,N,NaN,NaN,NaN,DJ,"988,002.00"
5958805,13643189,1171742,2019-05-28,CH,NaN,V,33,N,NaN,NaN,NaN,CH,"8,654,622.00"
5961554,13645938,1167439,2019-05-28,GB,NaN,H,38,N,NaN,NaN,NaN,GB,"67,886,011.00"
5961786,13646171,1168237,2019-05-28,DE,NaN,V,50,N,NaN,NaN,NaN,DE,"83,783,942.00"


In [ ]:
#elimino el country code que quedo de más
sociodemographic_merge.drop('2char country code', axis=1, inplace=True)

In [ ]:
sociodemographic_merge

,Unnamed: 0,pk_cid,pk_partition,country_id,region_code,gender,age,deceased,salary,Population_rEs,Region_name1,Population_country
0,0,1375586,2018-01-28,ES,29.00,H,35,N,"87,218.10","1,695,651.00",Málaga,"46,754,778.00"
1,1,1050611,2018-01-28,ES,13.00,V,23,N,"35,548.74","492,591.00",Ciudad Real,"46,754,778.00"
2,2,1050612,2018-01-28,ES,13.00,V,23,N,"122,179.11","492,591.00",Ciudad Real,"46,754,778.00"
3,3,1050613,2018-01-28,ES,50.00,H,22,N,"119,775.54","967,452.00",Zaragoza,"46,754,778.00"
4,4,1050614,2018-01-28,ES,50.00,V,23,N,NaN,"967,452.00",Zaragoza,"46,754,778.00"
...,...,...,...,...,...,...,...,...,...,...,...,...
5962919,13647304,1166765,2019-05-28,ES,50.00,V,22,N,"43,912.17","967,452.00",Zaragoza,"46,754,778.00"
5962920,13647305,1166764,2019-05-28,ES,26.00,V,23,N,"23,334.99","319,796.00",La Rioja,"46,754,778.00"
5962921,13647306,1166763,2019-05-28,ES,50.00,H,47,N,NaN,"967,452.00",Zaragoza,"46,754,778.00"
5962922,13647307,1166789,2019-05-28,ES,50.00,H,22,N,"199,592.82","967,452.00",Zaragoza,"46,754,778.00"


####Poblacion por region otros paises

In [ ]:
#Este es un nuevo df con las poblaciones de las regiones más pobladas de otros paises que no son españa
pop_region_oc

,country_id,Population_Roc,Region_name
0,GB,9002488,London
1,FR,12324261,Ill de france
2,DE,17925570,North Rhine-Westphalia
3,US,39613493,California
4,CH,1487969,Zurich
5,BR,46289333,Sao Paulo
6,BE,1869730,Flemish Region
7,VE,4957765,Zulia
8,MA,4270750,Casablanca
9,IE,2630720,Leinster


In [ ]:
#Aqui hago un merge para obtener una columna nueva en la tabla con el df anterior y uniendo por country_id
sociodemographic_merge= sociodemographic_merge.merge(pop_region_oc[['country_id', 'Region_name', 'Population_Roc']], how= 'left', left_on= 'country_id', right_on= 'country_id')

In [ ]:
#Genero una variable que me muestra los nulos de population_rEs
pop_rEs_nul = sociodemographic_merge['Population_rEs'].isna()

In [ ]:
reg_name_nul= sociodemographic_merge['Region_name1'].isna()

In [ ]:
sociodemographic_merge[sociodemographic_merge['pk_cid'] == 1005293]

,Unnamed: 0,pk_cid,pk_partition,country_id,region_code,gender,age,deceased,salary,Population_rEs,Region_name1,Population_country,Region_name,Population_Roc
78034,127081,1005293,2018-01-28,ES,28.00,V,6,N,"81,032.52","6,751,251.00",Madrid,"46,754,778.00",NaN,NaN
395991,1116878,1005293,2018-02-28,ES,28.00,V,6,N,"81,032.52","6,751,251.00",Madrid,"46,754,778.00",NaN,NaN
645539,1759395,1005293,2018-03-28,ES,28.00,V,6,N,"81,032.52","6,751,251.00",Madrid,"46,754,778.00",NaN,NaN
889915,2378356,1005293,2018-04-28,ES,28.00,V,6,N,"81,032.52","6,751,251.00",Madrid,"46,754,778.00",NaN,NaN
1062267,2648000,1005293,2018-05-28,ES,28.00,V,6,N,"81,032.52","6,751,251.00",Madrid,"46,754,778.00",NaN,NaN
1304155,3265537,1005293,2018-06-28,ES,28.00,V,6,N,"81,032.52","6,751,251.00",Madrid,"46,754,778.00",NaN,NaN
1575892,4337273,1005293,2018-07-28,ES,28.00,V,6,N,"81,032.52","6,751,251.00",Madrid,"46,754,778.00",NaN,NaN
1818268,4693333,1005293,2018-08-28,ES,28.00,V,6,N,"81,032.52","6,751,251.00",Madrid,"46,754,778.00",NaN,NaN
2315700,6037138,1005293,2018-09-28,ES,28.00,V,6,N,"81,032.52","6,751,251.00",Madrid,"46,754,778.00",NaN,NaN
2656684,6441192,1005293,2018-10-28,ES,28.00,V,6,N,"81,032.52","6,751,251.00",Madrid,"46,754,778.00",NaN,NaN


In [ ]:
2589312.000000

2589312.0

In [ ]:
sociodemographic_merge[sociodemographic_merge['pk_cid'] == 1009739]

,Unnamed: 0,pk_cid,pk_partition,country_id,region_code,gender,age,deceased,salary,Population_rEs,Region_name1,Population_country,Region_name,Population_Roc
82402,131493,1009739,2018-01-28,ES,28.00,V,13,N,"199,121.07","6,751,251.00",Madrid,"46,754,778.00",NaN,NaN
402628,1124655,1009739,2018-02-28,ES,28.00,V,13,N,"199,121.07","6,751,251.00",Madrid,"46,754,778.00",NaN,NaN
639146,1748059,1009739,2018-03-28,ES,28.00,V,13,N,"199,121.07","6,751,251.00",Madrid,"46,754,778.00",NaN,NaN
888580,2376878,1009739,2018-04-28,ES,28.00,V,13,N,"199,121.07","6,751,251.00",Madrid,"46,754,778.00",NaN,NaN
1064260,2649997,1009739,2018-05-28,ES,28.00,V,13,N,"199,121.07","6,751,251.00",Madrid,"46,754,778.00",NaN,NaN
1311826,3278154,1009739,2018-06-28,ES,28.00,V,13,N,"199,121.07","6,751,251.00",Madrid,"46,754,778.00",NaN,NaN
1577797,4339182,1009739,2018-07-28,ES,28.00,V,13,N,"199,121.07","6,751,251.00",Madrid,"46,754,778.00",NaN,NaN
1834817,4720225,1009739,2018-08-28,ES,28.00,V,13,N,"199,121.07","6,751,251.00",Madrid,"46,754,778.00",NaN,NaN
2323054,6045836,1009739,2018-09-28,ES,28.00,V,13,N,"199,121.07","6,751,251.00",Madrid,"46,754,778.00",NaN,NaN
2662138,6446649,1009739,2018-10-28,ES,28.00,V,13,N,"199,121.07","6,751,251.00",Madrid,"46,754,778.00",NaN,NaN


In [ ]:
sociodemographic_merge.head(50)

In [ ]:
sociodemographic_merge[sociodemographic_merge['pk_cid'] == 1018035]

,Unnamed: 0,pk_cid,pk_partition,country_id,region_code,gender,age,deceased,salary,Population_rEs,Region_name1,Population_country,Region_name,Population_Roc
25178,25326,1018035,2018-01-28,ES,28.00,H,7,N,"83,892.03","6,751,251.00",Madrid,"46,754,778.00",NaN,NaN
401407,1123424,1018035,2018-02-28,ES,28.00,H,7,N,"83,892.03","6,751,251.00",Madrid,"46,754,778.00",NaN,NaN
637936,1746840,1018035,2018-03-28,ES,28.00,H,7,N,"83,892.03","6,751,251.00",Madrid,"46,754,778.00",NaN,NaN
891728,2380183,1018035,2018-04-28,ES,28.00,H,7,N,"83,892.03","6,751,251.00",Madrid,"46,754,778.00",NaN,NaN
1055534,2641229,1018035,2018-05-28,ES,28.00,H,7,N,"83,892.03","6,751,251.00",Madrid,"46,754,778.00",NaN,NaN
1314694,3281024,1018035,2018-06-28,ES,28.00,H,7,N,"83,892.03","6,751,251.00",Madrid,"46,754,778.00",NaN,NaN
1613853,4404040,1018035,2018-07-28,ES,28.00,H,7,N,"83,892.03","6,751,251.00",Madrid,"46,754,778.00",NaN,NaN
1840455,4725863,1018035,2018-08-28,ES,28.00,H,7,N,"83,892.03","6,751,251.00",Madrid,"46,754,778.00",NaN,NaN
2307184,6028620,1018035,2018-09-28,ES,28.00,H,7,N,"83,892.03","6,751,251.00",Madrid,"46,754,778.00",NaN,NaN
2699512,6518011,1018035,2018-10-28,ES,28.00,H,7,N,"83,892.03","6,751,251.00",Madrid,"46,754,778.00",NaN,NaN


In [ ]:
sociodemographic_merge.head(80)

,Unnamed: 0,pk_cid,pk_partition,country_id,region_code,gender,age,deceased,salary,Population_rEs,Region_name1,Population_country,Region_name,Population_Roc
0,0,1375586,2018-01-28,ES,29.00,H,35,N,"87,218.10","1,695,651.00",Málaga,"46,754,778.00",NaN,NaN
1,1,1050611,2018-01-28,ES,13.00,V,23,N,"35,548.74","492,591.00",Ciudad Real,"46,754,778.00",NaN,NaN
2,2,1050612,2018-01-28,ES,13.00,V,23,N,"122,179.11","492,591.00",Ciudad Real,"46,754,778.00",NaN,NaN
3,3,1050613,2018-01-28,ES,50.00,H,22,N,"119,775.54","967,452.00",Zaragoza,"46,754,778.00",NaN,NaN
4,4,1050614,2018-01-28,ES,50.00,V,23,N,NaN,"967,452.00",Zaragoza,"46,754,778.00",NaN,NaN
5,5,1050615,2018-01-28,ES,45.00,H,23,N,"22,220.04","709,403.00",Toledo,"46,754,778.00",NaN,NaN
6,6,1050616,2018-01-28,ES,24.00,H,23,N,"295,590.36","451,706.00",León,"46,754,778.00",NaN,NaN
7,7,1050617,2018-01-28,ES,50.00,H,23,N,"113,316.66","967,452.00",Zaragoza,"46,754,778.00",NaN,NaN
8,8,1050619,2018-01-28,ES,20.00,H,24,N,NaN,"726,033.00",Gipuzkoa,"46,754,778.00",NaN,NaN
9,9,1050620,2018-01-28,ES,10.00,H,23,N,"113,194.98","389,558.00",Cáceres,"46,754,778.00",NaN,NaN


In [ ]:
#Con este loop relleno los nulos de population_rEs con los de population_Roc (para que todos los clientes tengan representada su región)
for i in range(0, len(sociodemographic_merge)):
  if pop_rEs_nul[i]:
    sociodemographic_merge.loc[i, 'Population_rEs'] = sociodemographic_merge['Population_Roc'][i]


In [ ]:
for i in range(0, len(sociodemographic_merge)):
  if reg_name_nul[i]:
    sociodemographic_merge.loc[i, 'Region_name1'] = sociodemographic_merge['Region_name'][i]

KeyboardInterrupt: ignored

In [ ]:
sociodemographic_merge[sociodemographic_merge['country_id'] != 'ES']

In [ ]:
sociodemographic_merge

In [ ]:
#Voy a renombrar la columna de region_rEs para que sea representativa con los datos añadidos de los demás países
sociodemographic_merge.rename(columns={"Population_rEs": "Population_region"},inplace=True)

In [ ]:
#borrar el region name
sociodemographic_merge.drop('Region_name',axis=1, inplace=True)

In [ ]:
sociodemographic_merge.rename(columns={"Region_name1": "Region_name"},inplace=True)

In [ ]:

sociodemographic_merge[sociodemographic_merge['country_id'] != 'ES'][['salary']].value_counts(dropna=False)

In [ ]:
2589312

In [ ]:
sociodemographic_merge[sociodemographic_merge['Population_region'] == 2589312]

In [ ]:
sociodemographic_merge[sociodemographic_merge['Population_region'] == 6751251]

In [ ]:
sociodemographic_merge[sociodemographic_merge['Population_region'] == 1044405]

In [ ]:
sociodemographic_merge[sociodemographic_merge['Population_region'] == 39613493]

#Variables numericas

###Age

In [ ]:
#la variable age no presenta nulos 
sociodemographic_merge['age'].isnull().sum()

In [ ]:
sociodemographic_merge['age'].value_counts()

In [ ]:
sociodemographic_merge['age'].min()

In [ ]:
sociodemographic_merge['pk_cid'].nunique()

In [ ]:
sociodemographic_merge[['age', 'pk_cid']] [sociodemographic_merge['age'] <= 17].groupby('pk_cid').describe() 

In [ ]:
sociodemographic_merge[['age', 'pk_cid']] [sociodemographic_merge['age'] <= 17].groupby('pk_cid').std()

In [ ]:
#clientes que tienen datos de edad ambiguos
i = 0
if sociodemographic_merge[['pk_cid']][sociodemographic_merge['age'] < 18].groupby('pk_cid').count().index is  cliente in sociodemographic_merge[['pk_cid']][sociodemographic_merge['age'] >= 18].groupby('pk_cid').count().index:

In [ ]:
sociodemographic_merge[['pk_cid']][sociodemographic_merge['age'] <= 17].groupby('pk_cid').count().index

In [ ]:
sociodemographic_merge['pk_cid'].unique()

In [ ]:
sociodemographic_merge['pk_cid'][sociodemographic_merge['age'] >= 17]

In [ ]:
sociodemographic_merge[sociodemographic_merge['pk_cid'] == 490491] ['age'].describe()

####Deseased

In [ ]:
sociodemographic_merge['deceased'].value_counts(dropna=False)

In [ ]:
#no tener en cuenta los muertos en compras activas 

#####Pasamos a booleanas 

In [ ]:
#De esta forma me quedaran en 0 aquellos que no murieron y en 1 los que si
sociodemographic_merge  = pd.get_dummies (data=sociodemographic_merge,columns=['deceased'])

del (sociodemographic_merge["deceased_N"]) 

In [ ]:
sociodemographic_merge.rename(columns={"deceased_S": "deseased"}, inplace=True)

###Salary. Ingresos brutos por unidad familiar

In [ ]:
sociodemographic_merge["salary"].value_counts(dropna = False)

In [ ]:
sociodemographic_merge["salary"].mean()

In [ ]:
#podemos ver quienes son los que tienen nulos en salary comparando con age?
sociodemographic_merge["age"].value_counts(dropna = False)

In [ ]:
sociodemographic_merge["salary"].describe()

In [ ]:
sociodemographic_merge['age'][sociodemographic_merge['salary'].isnull()].value_counts(normalize=True)

In [ ]:
sociodemographic_merge['salary'].fillna(sociodemographic_merge["salary"].median(), inplace=True)

In [ ]:
sociodemographic_merge.isnull().sum()

####Gender

In [ ]:
sociodemographic_merge['gender'].value_counts(dropna=False)

In [ ]:
#Verificamos que no hayan clientes que tengan el dato de genero en alguna otra fecha y podamos rellenarla con eso
sociodemographic_merge[sociodemographic_merge["gender"].isna()] ["pk_cid"].unique()

In [ ]:
476023

In [ ]:
sociodemographic_merge['gender'].fillna(sociodemographic_merge['gender'].mode(),inplace=True)

In [ ]:
sociodemographic_merge = pd.get_dummies (data=sociodemographic_merge,columns=['gender'])

del (sociodemographic_merge["gender_V"]) 

In [ ]:
sociodemographic_merge.info()

###Para reducir el uso de memoria voy a transformar las variables:

1.   Population_country y Population_region a int64
2.   Region_name y country_id a categorica
3.   A age puedo ponerle unit8 


In [ ]:
sociodemographic_merge['Population_country']= sociodemographic_merge['Population_country'].astype('int64')

In [ ]:
sociodemographic_merge['Population_region']= sociodemographic_merge['Population_region'].astype('int64')

In [ ]:
sociodemographic_merge['country_id']= sociodemographic_merge['country_id'].astype('category')

In [ ]:
sociodemographic_merge['age']= sociodemographic_merge['age'].astype('uint8')

In [ ]:
sociodemographic_merge.isnull().sum()

In [ ]:
sociodemographic_merge.info()

In [ ]:
sociodemographic_merge

In [ ]:
sociodemographic_merge['Population_Roc'].value_counts()

In [ ]:
sociodemographic_merge['Region_name'].isnull().sum()

In [ ]:
sociodemographic_merge.to_csv(r'/content/drive/MyDrive/CAPSTONE/sociodemographic_final.csv',index = False, sep=',')